In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [ ]:
path = (r'D:\Data Science\Projects\Next word prediction project\1661-0.txt')
text = open(path,encoding='utf-8').read().lower()
print('corpus length:', len(text))

In [ ]:
text

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)
print(words)

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
word_length = 5
prev_words = []
next_words = []
for i in range (len(words)-word_length):
    prev_words.append(words[i:i + word_length])
    next_words.append(words[i + word_length])
print(prev_words[0])
print(next_words[0])

In [ ]:
X = np.zeros((len(prev_words),word_length, len(unique_words)),dtype=bool)
Y = np.zeros((len(next_words),len(unique_words)),dtype=bool)
for i, prev_words in enumerate(prev_words):
    for j, each_word in enumerate(prev_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
print(X[0][0])

Building the Recurrent Neural network

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(word_length, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

Training the Next Word Prediction Model

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics = ['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history



In [ ]:
model.save('keras_next_word_model.h5')
pickle.dump(history,open("history.p",'wb'))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p","rb"))

Evaluating the Next Word Prediction Model

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')